## 요약
- XGB를 이용하여 여러가지 상황에서의 성능 확인
- hyperparameter 차이에 따른 성능 차이
  - grid search 이용해서 찾은 hyperparameter가 default일 때보다 성능이 다소 우위
- train set에 SalesPoint를 포함 했을 때와 포함하지 않았을 때
  - SalesPoint
- val, test set 전체에서의 성능과, train에 포함된 적 없는 도서에 대한 데이터로 제한했을 때의 성능
  - hyperparameter가 default일 때의 성능이 다소 우위

In [1]:
import os, natsort, re
from tqdm import tqdm
import time, random

In [2]:
from itertools import repeat, chain

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("fivethirtyeight")
# 이미지가 레이아웃 안으로 들어오도록 함
mpl.rcParams.update({"figure.constrained_layout.use": True})
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [4]:
#cd /content/drive/MyDrive/AI3_prjct2_aladin/

In [3]:
cd /content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/

[Errno 2] No such file or directory: '/content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/'
/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_book_price/research


In [4]:
# 로컬에서

plt.rc("font", family = "D2Coding")
plt.rcParams["axes.unicode_minus"] = False

In [5]:
PRJCT_PATH = '/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_usedbook/'
#PRJCT_PATH = '/content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/'
#PRJCT_PATH = '/content/drive/MyDrive/AI3_prjct2_aladin/aladin_usedbook/'
save_dir = 'processed/model_input'
dir_path = os.path.join(PRJCT_PATH,save_dir)
#dir_path = './'

In [6]:
ls

240710_crawling_step0.ipynb           240718_step0_by_js.ipynb
240711_crawling_step1.ipynb           240719_additional_eda.ipynb
240711_preprocess_bookinfo.ipynb      240719_simple_model_for_cropped.ipynb
240715_encoding_usedinfo.ipynb        240721_GridSearch_for_XGB.ipynb
240716_check_bookinfo.ipynb           240721_experiment_w_XGB.ipynb
240716_check_bookinfo2.ipynb          240721_hyperparameters_XGB.ipynb
240716_encoding_bookinfo.ipynb        241023_basic_model.ipynb
240717_simple_model_for_sample.ipynb  241023_preprocess_for_pred_rglr.ipynb
240717_split_and_scale.ipynb


In [7]:
import sys
sys.path.append(PRJCT_PATH)

In [8]:
RSLT_DIR = PRJCT_PATH + 'processed/'

bookinfo_name = 'bookinfo_ver{}.pkl'.format(0.75)
bookinfo_path = os.path.join(RSLT_DIR,bookinfo_name)

sys.path.append(PRJCT_PATH)
from module_aladin.file_io import load_pkl, save_pkl
from module_aladin.data_process import pd_datetime_2_datenum

from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.sequence import pad_sequences
import itertools
from sklearn.preprocessing import MinMaxScaler

2024-10-24 02:43:06.816031: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 02:43:07.472055: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 02:43:07.679526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-24 02:43:07.679625: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [9]:
def set_corpus_size(freq,size_feat,mode):
    # 입력받은 mode와 size_feat에 따라 size 크기 결정
    if mode == 'uniform':
        cond = freq['counts']>=freq['counts'].iloc[size_feat]
        size = np.sum(cond)
    elif mode =='ths':
        cond = freq[freq['counts'] > size_feat]
        size = np.sum(cond)
    else :
        if size_feat == None : size = len(freq)
        elif size_feat > len(data) : size = len(freq)
        else : size = size_feat
    return size

def make_encoding_by_freq(freq,null_val='[PAD]',size_feat=None,mode=None):
    #빈도수 기반 정수 인코딩 dict 만들기
    # freq : token 별 등장 빈도 (value_count), size_feat : size관련 변수(max_size, ths등), mode : size 결정 방법
    df_freq = pd.DataFrame(freq).T
    df_freq = df_freq.rename(columns={0:'token',1:'counts'})
    temp = df_freq.sort_values(by='counts',ascending=False)
    size = set_corpus_size(temp,size_feat,mode)
    temp = temp.iloc[:size]
    temp['val'] = np.arange(size)+1
    temp2 = temp.set_index('token').to_dict()
    map_token_encode = temp2['val']
    map_token_encode[null_val]=0
    return map_token_encode

def encode_tokens(map_token,x,oov=True):
    oov_val = len(map_token)+1 if oov else 0 
    return map_token[x] if x in map_token else oov_val

def make_author_encode_map(bookinfo,ths_author):
    pvtb = pd.pivot_table(data=bookinfo,index='Author',values='SalesPoint',aggfunc=np.sum)
    pvtb = pvtb.sort_values(by='SalesPoint',ascending=False)
    author_top_k= pvtb[pvtb['SalesPoint']>=ths_author].index
    encode_author = pd.DataFrame({'author' : author_top_k.values,'val':np.arange(1,len(author_top_k)+1)})
    encode_author = encode_author.set_index('author')
    return encode_author.to_dict()['val']

def make_publshr_encode_map(publshr_data,ths_publshr):
    stats = publshr_data.value_counts().sort_values(ascending=False)
    top_k_val = stats.iloc[ths_publshr]
    publshr_top_k = list(stats[stats >= top_k_val].index)
    return {
        publshr : n+1
        for n,publshr in enumerate(publshr_top_k)
    }
    

def make_store_encode_map(store_data):
    stores= store_data.value_counts().sort_values(ascending=False)
    return {
        place : n+1
        for n,place in enumerate(stores.index)
    }

In [10]:
file_name = 'bookinfo_ver{}.csv'.format(1.0)
file_path = os.path.join(RSLT_DIR,file_name)
bookinfo_raw = pd.read_csv(file_path)

In [11]:
bookinfo_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158084 entries, 0 to 158083
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BName       158083 non-null  object
 1   ItemId      158084 non-null  int64 
 2   BName_sub   9007 non-null    object
 3   Author      158084 non-null  object
 4   Publshr     158084 non-null  object
 5   Author_mul  158084 non-null  bool  
 6   Pdate       158084 non-null  int64 
 7   RglPrice    158084 non-null  int64 
 8   SlsPrice    158084 non-null  int64 
 9   SalesPoint  158084 non-null  int64 
 10  Category    158084 non-null  object
 11  Source      158084 non-null  object
dtypes: bool(1), int64(5), object(6)
memory usage: 13.4+ MB


In [12]:
cols = bookinfo_raw.columns.to_list()
x_idxs, y_idx = [0,2,3,4,5,6,9,10], 7
x_cols = [cols[i] for i in x_idxs]
y_col = cols[y_idx]

data_X, data_y  = bookinfo_raw[x_cols], bookinfo_raw[y_col]

In [13]:
from sklearn.model_selection import train_test_split

X_data, X_tst, y_data, y_tst = train_test_split(data_X,data_y,test_size=0.2,random_state=329)
X_trn, X_vld, y_trn, y_vld = train_test_split(X_data,y_data,test_size=0.2,random_state=329)

display(X_trn.columns)
display(X_trn.shape, y_trn.shape)
display(X_vld.shape, y_vld.shape)
display(X_tst.shape, y_tst.shape)

Index(['BName', 'BName_sub', 'Author', 'Publshr', 'Author_mul', 'Pdate',
       'SalesPoint', 'Category'],
      dtype='object')

(101173, 8)

(101173,)

(25294, 8)

(25294,)

(31617, 8)

(31617,)

In [14]:
data_dict = {
    'trn': {
        'X': X_trn,
        'y': y_trn
        },
    'vld':{
        'X': X_vld,
        'y': y_vld
        },
    'tst':{
        'X': X_tst,
        'y': y_tst
        
    }
}

In [15]:
mecab = Mecab()
tokenizer_basic = lambda x : mecab.morphs(x)
#apply tokenizer
cols_tknz = ['Category','BName','BName_sub']
cols_freq = ['Author','Publshr']

In [16]:
for mode, data in data_dict.items():
    bookinfo = data['X']
    for col in cols_tknz:
        bookinfo[col] = bookinfo[col].fillna('').apply(tokenizer_basic)
    data_dict[mode]['X'] = bookinfo

In [23]:
display(data_dict['trn']['X'].info())
display(data_dict['vld']['X'].info())
display(data_dict['tst']['X'].info())

<class 'pandas.core.frame.DataFrame'>
Index: 101173 entries, 96986 to 69788
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BName       101173 non-null  object
 1   BName_sub   101173 non-null  object
 2   Author      101173 non-null  object
 3   Publshr     101173 non-null  object
 4   Author_mul  101173 non-null  bool  
 5   Pdate       101173 non-null  int64 
 6   SalesPoint  101173 non-null  int64 
 7   Category    101173 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 6.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 25294 entries, 132734 to 43660
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BName       25294 non-null  object
 1   BName_sub   25294 non-null  object
 2   Author      25294 non-null  object
 3   Publshr     25294 non-null  object
 4   Author_mul  25294 non-null  bool  
 5   Pdate       25294 non-null  int64 
 6   SalesPoint  25294 non-null  int64 
 7   Category    25294 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 1.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 31617 entries, 146028 to 115151
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BName       31617 non-null  object
 1   BName_sub   31617 non-null  object
 2   Author      31617 non-null  object
 3   Publshr     31617 non-null  object
 4   Author_mul  31617 non-null  bool  
 5   Pdate       31617 non-null  int64 
 6   SalesPoint  31617 non-null  int64 
 7   Category    31617 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 2.0+ MB


None

In [24]:
#make encoding map
bookinfo = data_dict['trn']['X']
book_tknzed = bookinfo[cols_tknz].to_dict('series')
book_name, book_subname, category = book_tknzed['BName'], book_tknzed['BName_sub'],book_tknzed['Category']
tokens = np.array(list(itertools.chain(*book_name.values,*book_subname.values,*category.values)))
token_freq = np.unique(tokens,return_counts=True)

map_token_encode = make_encoding_by_freq(token_freq,size_feat=32000)
encode_1line =lambda x: list(map(lambda y : encode_tokens(map_token_encode,y),x))

In [25]:
#아래 ths 는 EDA 결과 제가 자의적으로 정한 내용
ths_author = int(np.round(len(book_name)/500)*75)
ths_publshr = int(np.round(len(book_name)/500)*5)

map_author_encode = make_author_encode_map(bookinfo[['Author','SalesPoint']],ths_author)
map_publshr_encode = make_publshr_encode_map(bookinfo['Publshr'],ths_publshr)

encode_maps = {
    'Author' : lambda x : encode_tokens(map_author_encode,x,oov=False),
    'Publshr' : lambda x : encode_tokens(map_publshr_encode,x,oov=False),
}

In [26]:
maxlens={
    'Category' : 5,
    'BName' : 30,
    'BName_sub' : 25
}
x_cols = ['BName', 'BName_sub', 'Author',
   'Author_mul', 'Publshr', 'Pdate', 'SalesPoint','Category']

book_cols = ['BName', 'BName_sub', 'Author', 'Author_mul', 'Publshr', 'Pdate',
       'RglPrice', 'SalesPoint', 'Category']
xcols_scalar = list(filter(lambda x : x not in cols_tknz,x_cols)) 

In [41]:
cols_else

['Author_mul', 'SalesPoint']

In [42]:
#encode X 
X_encoded=dict()
for mode,sample in tqdm(data_dict.items()):
    X_mode = sample['X']
    #padding and encoding
    encoded = pd.DataFrame(index=X_mode.index) 
    for col in cols_tknz :
        padded = pad_sequences(X_mode[col],padding='post',
                               maxlen=maxlens[col],
                               value='[PAD]',dtype=object)
        encoded[col] = list(np.apply_along_axis(encode_1line,0,padded))
    #concat encoded
    for col in tqdm(cols_freq):
        encoded[col] = X_mode[col].map(encode_maps[col])
    X_mode['Pdate'] = X_mode['Pdate'].astype(str)
#        date_format = '%Y-%m-%d'
    date_format = '%Y%m%d'
    encoded['Pdate']= pd.to_datetime(X_mode['Pdate'],format=date_format)
    encoded['Pdate']= pd_datetime_2_datenum(encoded['Pdate'])
    cols_else = list(filter(lambda x : x not in encoded.columns,x_cols))
    encoded[cols_else] = X_mode[cols_else]

    concat_tknzed =np.apply_along_axis(np.hstack,1,encoded[cols_tknz].values)
    x_scalar = encoded[xcols_scalar].values
    X = np.hstack((concat_tknzed,x_scalar))
    X_encoded[mode] = X    

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:05<00:00,  1.76s/it]


In [46]:
import copy

len_tknzed, len_scalar = sum(maxlens.values()), len(xcols_scalar)
scale_partition = [(0,len_tknzed)]+[(len_tknzed+i,len_tknzed+1+i) for i in range(len_scalar)]
scaler_list = [copy.deepcopy(MinMaxScaler()) for _ in scale_partition]
scalers = list(zip(scale_partition,scaler_list))


In [48]:
X_encoded['trn'].shape

(101173, 65)

In [49]:
scale_partition

[(0, 60), (60, 61), (61, 62), (62, 63), (63, 64), (64, 65)]

In [50]:
X_scaled = dict()
for mode in ['trn','vld','tst']:
    intermid = []
    for (cols,scaler) in scalers:
        sliced = X_encoded[mode][:,cols[0]:cols[1]]
        sliced = sliced.astype(np.float64)
        if mode == 'trn': sliced = scaler.fit_transform(sliced)
        else : sliced = scaler.transform(sliced)
        intermid.append(sliced)

    X_scaled[mode] = np.hstack(intermid)

In [51]:
data_type = 'prd_rglr'
strat=0
ver=1.0
dir_path = os.path.join(RSLT_DIR,'model_input')
for mode, x_scaled in X_scaled.items():
    save_pkl(dir_path,'{}.v{}_st-{}_X_{}.pkl'.format(data_type,ver,strat,mode),x_scaled)
for mode,sample in data_dict.items(): 
    save_pkl(dir_path,'{}.v{}_st-{}_y_{}.pkl'.format(data_type,ver,strat,mode),sample['y'].to_numpy())